<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-04-19 22:50:06


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
!pip install pyxirr
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=80cb296d4023ce42c98b21978c8aabf0049a95b85ecd72f130b76c0c6bbcff2c
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.6/515.6 kB 6.9 MB/s eta 0:00:00


In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-10*365)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ', 'EQUITAS']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
#cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'Conviction', 'Category', 'Stock', 'MedianPE']
#df_mypf = df_mypf[cols]
my_prospects = get_stock_ids(df_mypf)
df_stocks = df_mypf[df_mypf['Symbol'].isin(my_prospects)]

In [6]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO',
              'HDFCAMC', 'STARHEALTH', 'BAYERCROP', 'BIRLAMONEY', 'TATAELXSI', 'APEX', 'SPORTKING',
            'CASTROLIND', 'HONAUT']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [7]:
common_cols = ['Symbol', 'Target', 'Breakout', 'MedianPE', 'NSECap', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'Stock']
df_stocks = df_stocks[common_cols]

In [8]:

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))

      stock_df = yfin.Ticker(stock_symbol).history(period='10y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      #stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in stock_ids:
      #print(stock_id)
      #current_pe = get_current_pe(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp['Prev_Close'] = tmp['Close'].shift(1)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Criteria'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
      tmp['Strategy'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
      tmp['Target'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
      tmp['Breakout'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Breakout'].values[0]
      tmp['LatestQtr'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
      tmp['StarStock'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
      tmp['Conviction'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
      tmp['Category'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['Category'].values[0]
      tmp['Med_PE'] = df_mypf[(df_mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
      tmp['Curr_PE'] = current_pe
      tmp['Dev%_PE'] = round((tmp['Curr_PE'] - tmp['Med_PE'])*100/tmp['Med_PE'],2)
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
      #df_prec_dev = df_prec_dev.append(tmp)

  return df_prec_dev

stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

In [9]:
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [10]:
# portfolio stocks

def get_portfolio_features(df_common_features):

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = np.where(df_common_features['Breakout'].isnull(), df_common_features['Min'], df_common_features["Breakout"])

  #tmp_df = df_common_features[df_common_features['Stock'].isin(df_mypf[df_mypf['InPortfolio'] != 'NA'].Stock.values)]
  tmp_df = df_common_features
  print('qualified stocks: '+str(len(tmp_df['Stock'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Stock'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Stock'].unique())))

  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)

  return tmp_df

df_portfolio_features = get_portfolio_features(df_common_features)

qualified stocks: 138
with latest results: 109
still star stocks: 64


In [11]:
cols = ['Stock', 'Close', 'NTT',  'Dev%_200', 'Dev%_PE', 'RSI_14', 'NTT%', 'LTT%', 'Gained%',  'Criteria', 'Strategy','LatestQtr', 'StarStock', 'Conviction', 'Category']
df_portfolio_features = df_portfolio_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_portfolio_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [12]:
# av40
df_tmp = df_portfolio_features[df_portfolio_features['Criteria'].isin(['AV40'])].sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Stock,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
55,HDFCLIFE.NS,601.55,754.39,-4.97,-8.05,38.0,25.41,49.11,17.58,AV40,BTT,1,0,H,INSURANCE
97,PFIZER.NS,4130.20,5855.00,0.66,236.99,37.0,41.76,45.31,10.36,AV40,NTT,0,0,H,PHARMA
46,GILLETTE.NS,6445.45,7564.13,5.23,-16.33,44.0,17.36,39.32,52.60,AV40,BTT,0,1,H,FMCG
47,GLAXO.NS,1871.20,2466.80,7.97,-11.79,35.0,31.83,31.83,55.51,AV40,ATH,1,0,H,PHARMA
6,AKZOINDIA.NS,2406.25,3300.00,-5.49,-24.29,42.0,37.14,19.67,8.71,AV40,NTT,1,1,H,NaN
17,BAJAJHLDNG.NS,7835.70,9149.60,2.62,14.73,33.0,16.77,16.77,26.54,AV40,ATH,1,0,H,FINANCE
53,HCLTECH.NS,1447.50,1686.40,6.75,26.90,23.0,16.50,16.50,45.61,AV40,ATH,1,1,H,IT
98,PGHH.NS,15943.35,18360.87,-3.50,-13.78,44.0,15.16,15.16,19.92,AV40,ATH,0,0,H,FMCG
70,ITC.NS,424.75,485.03,-3.03,18.81,51.0,14.19,14.19,8.33,AV40,ATH,1,1,H,FMCG
1,ABBOTINDIA.NS,26128.00,27500.00,6.56,-2.81,38.0,5.25,13.05,25.79,AV40,NTT,0,1,H,PHARMA


In [13]:
# av40n
df_tmp = df_portfolio_features[df_portfolio_features['Criteria'].isin(['AV40N'])].sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Stock,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
72,JCHAC.NS,1119.35,3219.25,-1.55,1823.08,52.0,187.60,187.60,18.97,AV40N,ATH,0,0,M,AC
133,VIPIND.NS,513.95,740.05,-13.91,45.03,47.0,43.99,43.99,13.48,AV40N,ATH,1,0,M,MISC
132,VINATIORGA.NS,1649.35,1882.00,-5.32,14.05,63.0,14.11,41.49,12.25,AV40N,NTT,0,0,M,CHEMICALS
9,ANGELONE.NS,2688.10,3791.17,6.80,6.97,40.0,41.04,41.04,128.88,AV40N,ATH,1,1,M,FINANCE
27,CERA.NS,7103.05,9546.30,-11.92,-0.75,46.0,34.40,34.40,17.95,AV40N,ATH,1,1,M,CERAMICS
11,ASTRAZEN.NS,5189.70,6927.05,7.18,-19.76,46.0,33.48,33.48,66.20,AV40N,ATH,0,1,M,PHARMA
0,3MINDIA.NS,29660.50,37309.15,-3.63,-15.82,44.0,25.79,25.79,33.50,AV40N,ATH,1,1,M,MISC
44,FINCABLES.NS,970.45,1195.55,-2.95,24.06,50.0,23.20,23.20,25.06,AV40N,ATH,1,1,M,CABLES
24,CAPLIPOINT.NS,1311.70,1597.70,10.97,29.55,47.0,21.80,21.80,101.41,AV40N,ATH,1,1,M,PHARMA
68,ISEC.NS,710.65,841.40,3.90,-16.56,38.0,18.40,18.40,70.89,AV40N,ATH,1,1,M,FINANCE


In [14]:
# av200, avr and aar, exp50
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['AV40','AV40N'])].sort_values(by = 'LTT%', ascending=False)
df_tmp = df_tmp[df_tmp['Criteria'].notnull()]
data_table.DataTable(df_tmp, include_index=False)

,Stock,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
129,UPL.NS,487.55,NaN,-13.71,NaN,53.0,NaN,67.83,7.89,AAR,NTT,1,0,L,NaN
100,PNB.NS,128.25,90.00,44.10,11.45,50.0,-29.82,66.04,169.21,AVR,NTT,1,1,M,BANKS
91,NIACL.NS,218.95,278.00,19.64,65.81,38.0,26.97,63.36,119.39,AVR,NTT,1,0,M,INSURANCE
38,DREAMFOLKS.NS,513.10,815.64,-6.37,-12.44,53.0,58.96,58.96,20.37,EXP50,ATH,0,0,L,MISC
32,CUB.NS,154.40,166.00,11.88,NaN,64.0,7.51,55.09,16.09,AAR,NTT,0,1,L,BANKS
71,J&KBANK.NS,128.10,114.00,15.41,82.71,40.0,-11.01,38.77,157.85,AVR,NTT,1,1,M,BANKS
19,BIRLAMONEY.NS,117.10,141.00,24.57,-10.78,61.0,20.41,34.12,105.44,AAR,NTT,0,1,L,FINANCE
135,WSTCSTPAPR.NS,604.30,782.95,-6.02,NaN,42.0,29.56,29.56,32.39,AVR,ATH,0,0,M,PAPER
16,BAJAJHIND.NS,30.60,46.00,13.33,10652.69,44.0,50.33,27.12,128.36,AVR,NTT,1,0,M,SUGAR
103,PRAJIND.NS,508.75,644.60,-1.41,NaN,47.0,26.70,26.70,48.05,AAR,ATH,1,1,L,ENGINEERING


In [15]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'LTT%', ascending=False)
data_table.DataTable(df_portfolio_features, include_index=False)

,Stock,Close,NTT,Dev%_200,Dev%_PE,RSI_14,NTT%,LTT%,Gained%,Criteria,Strategy,LatestQtr,StarStock,Conviction,Category
136,YESBANK.NS,23.95,388.67,19.75,NaN,47.0,1522.84,1522.84,54.52,NaN,NaN,1,0,L,NaN
137,ZEEL.NS,142.85,588.15,-36.79,156.10,42.0,311.73,311.73,3.10,NaN,NaN,1,0,L,ENTERTAINMENT
72,JCHAC.NS,1119.35,3219.25,-1.55,1823.08,52.0,187.60,187.60,18.97,AV40N,ATH,0,0,M,AC
7,ALKYLAMINE.NS,2026.45,4589.29,-11.70,NaN,47.0,126.47,126.47,11.71,NaN,NaN,0,0,L,CHEMICALS
60,IBREALEST.NS,131.45,266.15,47.70,8749.56,56.0,102.47,102.47,140.53,NaN,NaN,1,0,L,REALTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,SAIL.NS,146.05,155.35,39.10,125.32,57.0,6.37,-1.40,82.22,AVR,BTT,1,0,M,STEEL
110,SBIN.NS,750.45,788.05,18.55,-26.29,49.0,5.01,-2.06,40.85,AVR,BTT,1,1,M,BANKS
54,HDFCAMC.NS,3708.45,3903.85,22.72,2.78,49.0,5.27,-2.92,119.19,AV40,BTT,1,1,H,FINANCE
74,JIOFIN.NS,370.10,378.40,41.26,NaN,60.0,2.24,-11.38,79.14,AVR,BTT,0,0,M,FINANCE
